In [1]:
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

In [2]:
gdf_building = gpd.read_file('../../data/processed/building/building_sample.geojson')
buildings_sindex = gdf_building.sindex

In [3]:
gdf_building.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US survey foot)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
def total_area_within_radius(building, radius):
    point = building['geometry'].centroid
    # Getting the index of candidate buildings within the radius centered at the building's centroid
    possible_matches_index = list(buildings_sindex.intersection(point.buffer(radius).bounds))
    # Retrieving the candidate buildings that are actually within the radius
    possible_matches = gdf_building.iloc[possible_matches_index]
    selected_buildings = possible_matches[possible_matches.intersects(point.buffer(radius))]
    # Calculating the total office, retail, and residential areas of the selected buildings
    total_office_area = selected_buildings['office_area'].sum()
    total_retail_area = selected_buildings['retail_area'].sum()
    total_residential_area = selected_buildings['residential_area'].sum()
    return total_office_area, total_retail_area, total_residential_area

gdf_building['office_within_450ft'], gdf_building['retail_within_450ft'], gdf_building['residential_within_450ft'] = zip(*gdf_building.progress_apply(lambda x: total_area_within_radius(x, 450), axis=1))

Applying function: 100%|██████████| 5733/5733 [00:12<00:00, 462.46it/s]


In [5]:
gdf_building.to_file('../../data/processed/building/building_sample.geojson', driver='GeoJSON')